In [7]:
! pip install pdfminer.six
! pip install pdfplumber
! pip install wordcloud

  Using cached wordcloud-1.8.2.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)


In [8]:
# load word embeddings
from utils import load_glove
import os
from urllib.request import urlretrieve
from main import lexranksentences,textrank,getalldata

# 6B.50,100,200,300 # 42B.300d


In [9]:
glove = 'glove.6B.100d.txt'
word_embeddings = load_glove(glove)

In [10]:
def crawlrep(assembly):
    wps = []
    wwwps=[]
    for i in range(1,570):
        wps.append(str(i).rjust(3,'0'))
    folder_name = 'a'+assembly+"_rev"
    #os.makedirs(folder_name)
    num_wp = 0
    for wp in wps:
        filename = 'wp_' + wp + '_en.pdf'
        # filename = 'wp_' + wp + '_rev1_en.pdf'
        #     filename = 'wp_' + wp + '_rev2_en.pdf'
        #     filename = 'wp_' + wp + '_corr1_en.pdf'
        #     filename = 'wp_' + wp + '_corr2_en.pdf'
        #     filename = 'wp_' + wp + '_rev1_corr1_en.pdf'
        try:
            urlretrieve("https://www.icao.int/Meetings/a41/Documents/WP/" + filename, folder_name + '/' + filename)
            #         urlretrieve("https://www.icao.int/Meetings/A40/Documents/" + filename, folder_name + '/' + filename)
            print('Downloading ' + filename + '......')
            data=getalldata(folder_name + '/' + filename, word_embeddings, glove_dim=100, num_sent=10,window_size=5, top_num=20)
            #sentences=lexranksentences(folder_name + '/' + filename, word_embeddings, glove_dim=100, num_sent=10)
            #keywords=textrank(pdf_path, window_size=5, top_num=20)
            wwwps.append({"wpid":wp,"wpidn":int(wp),"file":"https://www.icao.int/Meetings/a41/Documents/WP/" + filename,
                          "data":data})
            num_wp += 1
        except:
            pass
        print('-'*50 + '\nDownload ' + str(num_wp) + ' working papers in total.')
    return(wwwps)

In [11]:
x=crawlrep("41")

--------------------------------------------------
Download 1 working papers in total.
--------------------------------------------------
Download 2 working papers in total.
--------------------------------------------------
Download 3 working papers in total.
--------------------------------------------------
Download 4 working papers in total.
--------------------------------------------------
Download 5 working papers in total.
--------------------------------------------------
Download 6 working papers in total.
--------------------------------------------------
Download 7 working papers in total.
--------------------------------------------------
Download 8 working papers in total.
--------------------------------------------------
Download 9 working papers in total.
--------------------------------------------------
Download 10 working papers in total.
--------------------------------------------------
Download 11 working papers in total.
-----------------------------------------

In [12]:
x[0]

{'wpid': '001',
 'wpidn': 1,
 'file': 'https://www.icao.int/Meetings/a41/Documents/WP/wp_001_en.pdf',
 'data': {'sentences': ['Doc 7600/8, Standing Rules of Procedure of the Assembly of the International Civil Aviation Organization Doc 10140, Assembly Resolutions in Force (as of 4 October 2019) Appendix'],
  'keywords': ['assembly',
   'doc',
   'organization',
   'aviation',
   'civil',
   'international',
   'resolutions',
   'procedure',
   'force',
   'rules',
   'october',
   'standing',
   'appendix'],
  'opening': ['2',
   'Agenda Item 2: Approval of the Agenda',
   'PROVISIONAL AGENDA FOR THE 41ST SESSION OF THE ICAO ASSEMBLY'],
  'summary': ['The Provisional Agenda of the Assembly, as approved by the Council, is presented in the Appendix together with explanatory notes on each agenda item. Any items that the Council may decide to add will be notified immediately to Member States via a separate communication. Member States wishing to submit proposals for the addition of items s

In [13]:
import boto3
import json
s3 = boto3.client('s3')
filename='a41data.json'




In [14]:
with open(filename, 'w') as outfile:
    json.dump(x, outfile)

In [15]:
s3.upload_file(filename, "isicaodata", "Deliveries/A41Data/"+filename,ExtraArgs={'ACL': 'public-read'})
